# Cvičenie 9: Actor-critic architektúry


Na predošlých cvičeniach sme videli, ako fungujú rôzne algoritmy učenia posilňovaním a ako dokážu pracovať so spojitým stavovým priestorom. Cieľom dnešného cvičenia je predstaviť architektúry actor-critic využívajúce neurónové siete na prácu so spojitým priestorom akcií.

## Actor-critic algoritmy

Actor-critic algoritmy sú založené na politike, na rozdiel od Q-sietí teda namiesto toho, aby aproximovali hodnotu dvojíc stav-akcia a na základe tohto odhadu aktualizovali svoju politiku aktualizujú priamo politiku. Toto zlepšenie je možné na základe skóre, ktoré vyjadruje vhodnosti danej politiky na vyriešenie problému. V actor-critic architektúrach sa používajú oba prístupy a to tak, že celkový agent obsahuje dva modely.

**Actor** je zodpovedný za výber akcií cez funkciu politiky $\pi(s, a; \theta)$, pričom **critic** vyhodnocuje túto politiku cez aproximáciu hodnotovej funkcie $\hat{q}(s, a; w)$. Vyhodnocovanie zvyčajne prebieha cez temporal difference metódu. Critic aproximuje hodnotovú funkciu na základe príkladov, ktoré sa pozbierajú počas interakcie s prostredím. Následne táto zmena je použitá na aktualizáciu parametrov politiky actora tak, aby výkon sa zlepšil cez lepší výber akcií. Konvergencia je zabezpečená malým krokom v gradiente politiky, čo znamená, že zmena v hodnotovej funkcii spôsobí iba malú zmenu v politike.

<img src="lab09/actor-critic.jpg" width="750">
<p style="text-align: center;">Zdroj: Grondman, I., Busoniu, L., Lopes, G. A., and Babuska, R. (2012). A Survey of Actor-Critic Reinforcement Learning: Standard and Natural Policy Gradients. IEEE Transactions on Systems, Man, and Cybernetics, Part C (Applications and Reviews), 42(6):1291–1307.</p>

Celková interakcia teda prebieha nasledovne:

1. na základe aktuálneho stavu prostredia $s_t$, actor vyberie akciu $a_t$;
2. prostredie zareaguje na vykonanie akcie zmenou svojho stavu na $s_{t+1}$, z tejto zmeny sa vypočíta odmena $r_{t+1}$;
3. odmena je spracovaná criticom, ktorý ju použije na výpočet kvality aktuálnej politiky actora cez adaptáciu aktuálnych odhadov hodnotovej funkcie;
4. actor sa aktualizuje po niekoľkých krokoch (čiarkovaná čiara).

## Trénovanie modelov actora a critica

Trénovanie prebieha teda v dvoch fázach, pričom častejšie sa aktualizuje critic na základe vzorca:

$$\delta w = \beta (r_{t+1} + \gamma \cdot \hat{q}_w (s_{t+1}, a_{t+1}) - \hat{q}_w (s_{t}, a_{t})) \bigtriangledown_w \hat{q}_w (s_{t}, a_{t}))$$

kde $\beta$ je učiaci parametere critica, $\hat{q}_w (s_{t}, a_{t})$ je odhadovaná hodnota dvojice stav-akcia, $r_{t+1} + \gamma \cdot \hat{q}_w (s_{t+1}, a_{t+1}) - \hat{q}_w (s_{t}, a_{t})$ je temporal difference chyba, a $\bigtriangledown_w \hat{q}_w (s_{t}, a_{t}))$ je gradient hodnotovej funkcie.

Actor sa aktualizuje pomocou vlastného učiaceho parametra $\alpha$:

$$\delta \theta = \alpha \bigtriangledown_{\theta}(log \ \pi_\theta(a_t|s_t))  \hat{q}_w (s_{t}, a_{t}))$$

## Zisková funkcia

Vzhľadom na veľkú variabilitu hodnôt stavov a akcií (hodnota môže byť zo širokého intervalu), namiesto štandardnej hodnotovej funkcie sa často použije zisková funkcia $A(s_t, a_t)$. Namiesto toho teda, aby sme vypočítali očakávanú hodnotu každej akcie, zisk vyjadruje vylepšenie pri výbere danej akcie oproti iným akciám v tom istom stave. Ak $A(s_t, a_t) > 0$, znamená to, že akcia je lepšia ako priemerná akcia v danom stave, a gradient by teda mal smerovať k tejto akcii. Záporný zisk znamená, že akcia je podpriemerná, a politika by teda mala upredňostňovať iné akcie. Ak poznáme očakávanú hodnotu stavu $v(s_t)$, zisk vieme vyjadriť ako $A(s_t, a_t) = Q(s_t, a_t) - v(s_t)$, resp. $A(s_t, a_t) = r_{t+1} + \gamma \cdot v(s_{t+1}) - v(s_t)$. Druhý vzorec vieme použiť ako aproximáciu TD-chyby do vzorca pre aktualizáciu váh kritika.

## Actor-critic algoritmy

Dva základné populárne algoritmy typu actor-critic sú **Advantage Actor-Critic** (A2C) a **Asynchronous Advantage Actor-Critic** (A3C). V A2C máme jednu globálnu sieť a niekoľko kópií agentov, ktoré komunikujú s vlastnou inštanciou prostredia. Agenti sú natrénovaní najprv nezávisle, a následne sa spriemeruje ich gradient pre trénovanie globálnej siete. Takéto nastavenie umožní paralelizáciu na grafických kartách a lepšie podporuje väčšie batche.

V algoritme A3C sú agenti trénovaní paralelne, čo spôsobí skrátený čas trénovania. Globálna sieť je aktualizovaná asynchrónne, čo môže spôsobiť, že niektorí agenti použijú inú verziu politiky, a to môže spôsobiť suboptimálnu agregovanú zmenu siete.

## Spojitý priestor akcií

Príkladom algoritmu actor-critic s podporou spojitého priestoru akcií je `DDPG`, teda *deep deterministic policy gradient*. Vstupom pre sieť actora je reprezentácia stavu, a výstupom je hodnota, ktorá určuje akciu. Napríklad, ak máme úlohu riadenia vozidla, a jedna z akcií nastavuje rýchlosť vozidla v intervale 0 - 130, tak actor bude generovať číslo z tohto intervalu. Critic ako vstup berie stavovú reprezentáciu, a akciu agenta, teda výstup actora. Výstupom critic siete je jedno číslo, a to očakávaná Q-hodnota danej akcie pri danom stave prostredia.

Algoritmus trénovania nájdete nižšie.

<img src="lab09/ddpg.jpg" width="750">

Fungovanie `DDPG` agenta si ukážeme na prostredí [Pendulum](https://gymnasium.farama.org/environments/classic_control/pendulum/). [Stiahnite si ukážkovú implementáciu](lab09/ddpg.py), a prediskutujte kód a jednotlivé kroky potrebné k dosiahnutiu riešenia.

**Úloha:** Na základe ukážkového riešenia skúste vytvoriť agenta s algoritmom DDPG, ktorý sa naučí navigovať v spojitom Gridworld prostredí cez spojité akcie. Ako akciu použijete uhol posunu z intervalu $<-\pi, \pi>$.